In [21]:
import numpy as np
import pandas as pd

In [84]:
df = pd.read_csv("AutoLoanData.csv")
df.columns

Index(['Tier', 'Primary_FICO', 'State', 'Type', 'Apply_Date', 'Approve_Date',
       'Fund_Date', 'Term', 'New_Rate', 'Used_Rate', 'Amount_Approved',
       'Previous_Rate', 'CarType', 'Competition_rate', 'SAS_Apply_Date',
       'SAS_Approve_Date', 'apply', 'rate', 'onemonth', 'NewUsed',
       'SAS_Fund_Date', 'buffer', 'enddate', 'days', 'weeks', 'months',
       'termclass', 'rate1', 'rel_compet_rate', 'mp', 'mp_rto_amtfinance',
       'partnerbin', 'CarType_id'],
      dtype='object')

In [29]:
df1 = df[['apply', 'Primary_FICO', 'Amount_Approved', 'onemonth', 'Competition_rate', 'mp', 'Term']].copy(deep=True)
T = df1.shape[0]
df1.head()
# apply: binary feedback
# Primary_FICO - Competition_rate: features

,apply,Primary_FICO,Amount_Approved,onemonth,Competition_rate,mp,Term
0,0,695,35000.0,1.8388,6.25,604.984462,72
1,0,751,40000.0,1.8388,5.65,763.861873,60
2,0,731,18064.0,1.8388,5.65,344.960022,60
3,0,652,15415.0,1.8388,6.25,277.787156,72
4,0,730,32000.0,1.8388,5.65,744.061380,48


In [ ]:
## compute the price column
libor = 1.44 / 100 / 12
ro = 1 / (1 + libor)
df1['price'] = 0.0
for t in range(T):
    if np.mod(t, 10000) == 0:
        print(t, end = ' ')
    mp_t = df1.iloc[t]['mp']
    Term_t = df1.iloc[t]['Term']
    loan_t = df.iloc[t]['Amount_Approved']
    df1.loc[[t], ['price']] = mp_t * (ro - ro ** (Term_t + 1)) / (1 - ro) - loan_t 

In [80]:
## normalization
for feature in ['Primary_FICO', 'Amount_Approved', 'onemonth', 'Competition_rate', 'price']:
    colarr = np.array(df1.loc[:, [feature]])
    colmax = np.max(colarr)
    col_normalized = colarr / colmax
    df1.loc[:, [feature]] = col_normalized

In [81]:
df2 = df1[['apply', 'Primary_FICO', 'Amount_Approved', 'onemonth', 'Competition_rate', 'price']]

In [82]:
df2.head()

,apply,Primary_FICO,Amount_Approved,onemonth,Competition_rate,price
0,0,0.813817,0.35000,0.864504,0.968992,0.245394
1,0,0.879391,0.40000,0.864504,0.875969,0.153507
2,0,0.855972,0.18064,0.864504,0.875969,0.069324
3,0,0.763466,0.15415,0.864504,0.968992,0.136671
4,0,0.854801,0.32000,0.864504,0.875969,0.098272


In [85]:
df2.to_csv('Loan_preprocessed.csv')